In [80]:
from PIL import Image, ImageDraw
import argparse
import os
import glob
import numpy as np

In [81]:
RAW_WIDTH = 640
RAW_HEIGHT = 480

In [82]:
# Dataset operations

In [83]:
def read_label_file(path):
    with open(path) as f:
        xys = []
        has_nan = False
        for l in f:
            x, y = map(float, l.split())
            # XXX some bounding boxes has nan coordinates
            if np.isnan(x) or np.isnan(y):
                has_nan = True
            xys.append((x, y))
            if len(xys) % 4 == 0 and len(xys) / 4 >= 1:
                if not has_nan:
                    yield xys[-4], xys[-3], xys[-2], xys[-1]
                has_nan = False

In [84]:
def convert_pcd(path):
    with open(path) as f:
        # move pass the header
        # http://pointclouds.org/documentation/tutorials/pcd_file_format.php
        for _ in range(11):
            f.readline()
            pass
        im = np.zeros((RAW_HEIGHT, RAW_WIDTH), dtype='f4')
        for l in f:
            d, i = l.split()[-2:]
            d = float(d)
            i = int(i)
            x = i % RAW_WIDTH
            y = i % RAW_HEIGHT
            im[y, x] = max(0., d)
        return im

In [85]:
partial_path = os.getcwd().rsplit('/', 1)[0] + '/data/[0-9]*/'

In [86]:
partial_path

'/home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/data/[0-9]*/'

In [87]:
parser = argparse.ArgumentParser()
parser.add_argument('partial_path')

#args = parser.parse_args()
args = parser.parse_args([partial_path])

In [88]:
args

Namespace(partial_path='/home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/data/[0-9]*/')

In [89]:
green = (0, 255, 0)
red = (255, 0, 0)

In [90]:
def draw_box(p1, p2, p3, p4, draw, color):
    ps = [p1, p2, p3, p4]
    for i in range(4):
        x1, y1 = ps[i]
        x2, y2 = ps[(i+1) % 4]
        if i % 2 == 1:            
            draw.line((x1, y1, x2, y2), fill=color)        
        else:            
            draw.line((x1, y1, x2, y2))    

In [91]:
filename_format = '%s-%03i'
for path in glob.glob('%s/pcd*[0-9].txt' % args.partial_path):   
    sample_id = path[-len('1234.txt'):-len('.txt')] 
    depth_data = convert_pcd('%s' % path)
    depth_img = Image.new('F', (640, 480))
    # strange scale in the data
    depth_img.putdata(depth_data.flatten() * 1e40)
    depth_img.show()
    
    with Image.open(path[:-len('.txt')]+'r.png') as img:                   
        draw = ImageDraw.Draw(img)
        for i, box in enumerate(read_label_file(path[:-len('.txt')]+'cpos.txt')):              
            newimg=path[:-len('.txt')]+'cpos.png'
            draw_box(box[0], box[1], box[2], box[3], draw, green)
            draw.text(box[0], str(i), fill=green)
            img.save(newimg, format='PNG')

        for i, box in enumerate(read_label_file(path[:-len('.txt')]+'cneg.txt')):
            newimg=path[:-len('.txt')]+'cneg.png'
            draw_box(box[0], box[1], box[2], box[3], draw, red)
            draw.text(box[0], str(i), fill=red)
            img.save(newimg, format='PNG')

/home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/data/01/pcd0182r.png
/home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/data/01/pcd0163r.png
/home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/data/01/pcd0193r.png
/home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/data/01/pcd0140r.png
/home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/data/01/pcd0109r.png
/home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/data/01/pcd0159r.png
/home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/data/01/pcd0177r.png
/home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/data/01/pcd0129r.png
/home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/data/01/pcd0186r.png
/home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/data/01/pcd0147r.png
/home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/data/01/pcd0120r.png
/home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/data/01/pcd0141r.png
/hom